In [1]:
from datetime import datetime 
import dateutil.relativedelta
import time
import csv
from collections import OrderedDict

In [16]:
from search_api import articleAPI
api = articleAPI("S49SxPkHv9vcW0RyJqVyM7fCASbkThWL")

In [19]:
articles = api.search(q="Tropical Cyclones And Global Warming",\
                page=10,\
                begin_date= 20000101,\
                sort = "newest")

In [20]:
articles

{'status': 'OK',
 'copyright': 'Copyright (c) 2020 The New York Times Company. All Rights Reserved.',
 'response': {'docs': [{'abstract': 'Ten climate experts have defined “the main hurricane problem facing the United States.”',
    'web_url': 'https://www.nytimes.com/2006/07/25/science/earth/25coast.html',
    'snippet': 'Ten climate experts have defined “the main hurricane problem facing the United States.”',
    'lead_paragraph': 'Ten climate experts who are sharply divided over whether global warming is intensifying hurricanes say that this question, a focus of Congressional hearings, news reports and the recent Al Gore documentary, is a distraction from “the main hurricane problem facing the United States.”',
    'print_section': 'F',
    'print_page': '2',
    'source': 'The New York Times',
    'multimedia': [],
    'headline': {'main': 'Climate Experts Warn of More Coastal Building',
     'kicker': None,
     'content_kicker': None,
     'print_headline': 'Climate Experts Warn 

In [5]:
def time_calculation():
    timeNow = datetime.now()
    timeNow2=timeNow.strftime("%Y-%m-%d")
    d = datetime.strptime(timeNow2, "%Y-%m-%d")
    d2 = d - dateutil.relativedelta.relativedelta(years=20)
    #如果是要搜尋一個月內的新聞，則輸入(months=1)
    d3 = d2.strftime("%Y%m%d")
    return(d3)


'20001126'

In [185]:
def getting_articles(a):
    allArticles = []
    allSubjects = {}
    for i in range(0,100):                   #api限制page最高只能到100
        articles = api.search(q=a,\
            fq={"source":["The New York Times","Reuters","AP"]},\
            page=i,\
            begin_date=time_calculation(),  #將時間函式產出的結果套入 
            sort = "newest")

In [12]:
def compile_articles(articles):
    compiled= []
    for i in articles["response"]["docs"]:
        time.sleep(0.5)  
        dictA = {}
        dictA["headline"]=i["headline"]["main"]
        if i["snippet"] is not None:
            dictA["snippet"] = i["snippet"]
        dictA["date"]=i["pub_date"][0:10]
        dictA["link"] = i["web_url"]
        compiled.append(dictA)
    return(compiled)

In [13]:
def getting_articles(a):
    allArticles = []
    allSubjects = {}
    for i in range(0,100):
        articles = api.search(q=a,\
            page=i,\
            begin_date=time_calculation(),\
            sort = "newest")
        if articles["response"]["docs"] != []: 
            for j in articles["response"]["docs"]:
                time.sleep(0.5)
                for x in range(0,len(j['keywords'])):
                    if 'subject' in j['keywords'][x]['name']:
                        if j['keywords'][x]['value'] not in allSubjects.keys():
                            allSubjects[j['keywords'][x]['value']] = 1
                        else:
                            allSubjects[j['keywords'][x]['value']] = \
                            allSubjects[j['keywords'][x]['value']] + 1
            articles = compile_articles(articles)                                                               
            allArticles = allArticles + articles
        else:
            break
    row = []
    allSubjects = OrderedDict(sorted(allSubjects.items(), key=lambda d: d[1],reverse = True))     
    for i in range(len(allSubjects)):
        B= {}
        B["Subjects"] = list(allSubjects.keys())[i]
        B["times"] = allSubjects[list(allSubjects.keys())[i]]
        row.append(B)
    intoCsv(allArticles,row)  


In [14]:
def intoCsv(a,b):
    with open("theDoc.csv","w",encoding='utf-8') as fOut:
        csvOut = csv.DictWriter(fOut, 
        ["date","headline","snippet","link"])
        csvOut.writeheader()
        csvOut.writerows(a)
    
    with open("theStatistics.csv","w",encoding='utf-8') as fOut:
        csvOut = csv.DictWriter(fOut,["Subjects","times"])
        csvOut.writeheader()
        csvOut.writerows(b)

In [15]:
getting_articles('Tropical Cyclones And Global Warming')